<a href="https://colab.research.google.com/github/padmathanumoorthy/Amazon-Bedrock-And-StabilityAI-Project/blob/main/Text-To-Image-Stable-Diffusion-XL-10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
%%sh

# create virtual python environment
python3 -m pip install pip -Uq



In [2]:
%%sh

# install requirements
python3 -m pip install boto3 -Uq
python3 -m pip install botocore -Uq
python3 -m pip install ffmpeg-python -Uq
python3 -m pip install Pillow -Uq





   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 12.2 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.2.0 which is incompatible.


In [4]:
import os
from google.colab import userdata
userdata.get('AWS_ACCESS_KEY_ID')
userdata.get('AWS_SECRET_ACCESS_KEY')



'nLIvY9CilPEo0QKSt8tb1f8SK40hF8GUcGUenOzt'

In [11]:
%%sh

# install ffmpeg on mac using homebrew
# https://phoenixnap.com/kb/ffmpeg-mac

#brew update
# brew upgrade # this is optional (note: will upgrade all packages and could take awhile)
#brew install ffmpeg
python3 -m pip install ffmpeg -Uq

mkdir: cannot create directory ‘/content/source_videos/’: File exists


In [25]:
%%sh

mkdir /content/source_videos/
mkdir /content/source_frames/
mkdir /content/generated_frames/
mkdir /content/generated_images/

mkdir /content/source_frames/demo1/
mkdir /content/source_frames/demo2/
mkdir /content/source_frames/demo3/
mkdir /content/source_frames/demo4/
mkdir /content/generated_frames/demo1/
mkdir /content/generated_frames/demo2/
mkdir /content/generated_frames/demo3/
mkdir /content/generated_frames/demo4/

ls /content/

bedrock-venv
generated_frames
generated_images
sample_data
source_frames
source_videos


mkdir: cannot create directory ‘/content/source_videos/’: File exists
mkdir: cannot create directory ‘/content/source_frames/’: File exists
mkdir: cannot create directory ‘/content/generated_frames/’: File exists
mkdir: cannot create directory ‘/content/source_frames/demo1/’: File exists
mkdir: cannot create directory ‘/content/source_frames/demo2/’: File exists
mkdir: cannot create directory ‘/content/source_frames/demo3/’: File exists
mkdir: cannot create directory ‘/content/source_frames/demo4/’: File exists
mkdir: cannot create directory ‘/content/generated_frames/demo1/’: File exists
mkdir: cannot create directory ‘/content/generated_frames/demo2/’: File exists
mkdir: cannot create directory ‘/content/generated_frames/demo3/’: File exists
mkdir: cannot create directory ‘/content/generated_frames/demo4/’: File exists


In [16]:
# Amazon Bedrock Model ID used throughout this notebook
# Model IDs: https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html#model-ids-arns
MODEL_ID = "stability.stable-diffusion-xl-v1"

In [17]:
# Location of visual contents used throughout this notebook
SOURCE_IMAGES = "/content/source_images"
SOURCE_VIDEOS = "/content/source_videos"
SOURCE_FRAMES = "/content/source_frames"

GENERATED_IMAGES = "/content/generated_images"
GENERATED_FRAMES = "/content/generated_frames"
GENERATED_VIDEOS = "/content/generated_videos"

In [47]:
# Modified version of AWS example code: https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-diffusion-1-0-text-image.html#model-parameters-diffusion-1-0-code-example
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: Apache-2.0
"""
Shows how to generate an image with SDXL 1.0 (on demand).
"""

import base64
import io
import json
import logging
import boto3
from PIL import Image
import time

from botocore.exceptions import ClientError

# There is a 77-token limit for prompts for the SXDL 1.0 model

# Phoenix prompt inspiration: https://civitai.com/images/3030250
# preset style: fantasy-art, cfg_scale: 15, sampler: K_DPMPP_2M, samples: 1, seed: 72746264, steps: 25
POSITIVE_PROMPT = "Pixar: ironman, cute modern disney style, Pixar 3d portrait, ultra detailed, gorgeous, 3d zbrush, trending on dribbble, 8k render"
NEGATIVE_PROMPT = ""

# Phoenix prompt inspiration: https://civitai.com/images/3030250
# preset style: fantasy-art, cfg_scale: 15, sampler: K_DPMPP_2M, samples: 1, seed: 72746264, steps: 25
# POSITIVE_PROMPT = "mythical beast, Phoenix, big eyes, sharp beak, long flowing feathers, bright colors, outdoors, perched on branch, no_humans, fire,scenery, cinematic lighting, strong contrast, high level of detail, best quality, masterpiece, best quality, ultra-detailed, masterpiece, hires, 8k"
# NEGATIVE_PROMPT = "(worst quality:1.5), (low quality:1.5), (normal quality:1.5), lowres, bad anatomy, bad hands, watermark, moles, toe, bad-picture-chill-75v, realisticvision-negative-embedding, (monochrome:1.5), (grayscale:1.5), (bad proportions:1.3)"

# Samurai prompt inspiration: https://prompthero.com/prompt/accdc184a10
# preset style: fantasy-art, cfg_scale: 15, sampler: K_DPMPP_2M, samples: 1, seed: 72746264, steps: 40
# POSITIVE_PROMPT = "digital wallpaper portrait of a samurai in black armor, highly detailed, at night, dramatic cinematic lighting, rainy, reflections in puddles, dramatic composition, emotionally profound, leaving an indelible and haunting impression on psyche, very intricate, by Milo Manara and Russ Mills"
# NEGATIVE_PROMPT = "(deformed iris, deformed pupils), text, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, (extra fingers:1.5), (mutated hands:1.5), poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, (fused fingers), (too many fingers), long neck, camera, name, signature, watermark, logo, autograph, trademark, cut off, censored, bad anatomy, bad body, bad face, bad teeth, deformities, (boring, uninteresting:1.1)"

# Cat prompt inspiration: https://civitai.com/images/6826034
# No preset style, cfg_scale: 12, sampler: K_DPMPP_2M, samples: 1, seed: 72746264, steps: 25
# POSITIVE_PROMPT = "amazing quality, masterpiece, best quality, hyper detailed, ultra detailed, UHD, perfect anatomy, magic world, (kitten and fish:1.4), fish in the air, spell magic to get fresh fish as food,( fish jumping from magic book:1.3), energy flow, a full body of a cute kitten, kawaii, wearing witches robe, witches hat, holding magic book, magic book on one hand, spell magic, hkmagic, extremely detailed, glowneon, glowing"
# NEGATIVE_PROMPT = "FastNegativeV2, watermark, signature, worst quality, low quality, normal quality, lowres, simple background, inaccurate limb, extra fingers, fewer fingers, missing fingers, extra arms, (extra legs:1.3), inaccurate eyes, bad composition, bad anatomy, error, extra digit, fewer digits, cropped, low res, worst quality, low quality, normal quality, jpeg artifacts, extra digit, fewer digits, trademark, watermark, artist's name, username, signature, text, words"

# Robot prompt inspiration: https://civitai.com/images/6935869
# preset style: fantasy-art, cfg_scale: 15, sampler: K_DPMPP_2M, samples: 1, seed: 72746264, steps: 40
# POSITIVE_PROMPT = "masterpiece, best quality, cinematic film still, realistic, portrait, solo, white mecha robot, cape, science fiction, torn clothes, glowing, standing, robot joints, mecha, armor, cowboy shot, (floating cape:1.3), intense sunlight, silver dragonborn, outdoors, landscape, nature highres, 4k, 8k, intricate detail, cinematic lighting, amazing quality, wallpaper"
# NEGATIVE_PROMPT = "nipples, (low quality, worst quality:1.4), cgi, text, signature, watermark, extra limbs"

# Batman prompt inspiration: https://medium.com/phygital/top-40-useful-prompts-for-stable-diffusion-xl-008c03dd0557
# No preset style, cfg_scale: 5, sampler: K_DPMPP_2M, samples: 1, seed: 72746264, steps: 35
# POSITIVE_PROMPT = "batman, cute modern disney style, Pixar 3d portrait, ultra detailed, gorgeous, 3d zbrush, trending on dribbble, 8k render"
# NEGATIVE_PROMPT = ""

# Dragon prompt inspiration: https://civitai.com/images/7029232
# No negative prompt, preset style: anime, cfg_scale: 15, sampler: K_DPMPP_2M, samples: 1, seed: 1121313, steps: 25
# POSITIVE_PROMPT = "silk screen art, (ukiyo-e:1.1), (painted by Kim Hong-do:1.1), sharp oriental dragon, green and gold with a bit of blue, katsuhiro otomo, line art, glitch art, hergé"
# NEGATIVE_PROMPT = "(deformed iris, deformed pupils), text, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, (extra fingers:1.5), (mutated hands:1.5), poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, (fused fingers), (too many fingers), long neck, camera, name, signature, watermark, logo, autograph, trademark, cut off, censored, bad anatomy, bad body, bad face, bad teeth, deformities, (boring, uninteresting:1.1)"

# Autumn girl prompt inspiration: https://civitai.com/images/6840704
# v1 No negative prompt, no preset style, cfg_scale: 5, sampler: K_DPMPP_2M, samples: 1, seed: 123456, steps: 30
# v2 No negative prompt, no preset style, cfg_scale: 15, sampler: K_DPMPP_2M, samples: 1, seed: 123456, steps: 25
# POSITIVE_PROMPT = "mysterious silhouette forest woman, by Minjae Lee, Carne Griffiths, Emily Kell, Geoffroy Thoorens, Aaron Horkey, Jordan Grimmer, Greg Rutkowski, amazing depth, masterwork, surreal, geometric patterns, intricately detailed, bokeh, perfect balanced, deep fine borders, artistic photorealism , smooth, great masterwork by head of prompt engineering"
# NEGATIVE_PROMPT = "boring,text,signature,logo,watermark,low quality, bad quality, loose artifacts, grainy, blurry, long neck, closed eyes, face jewellery"


class ImageError(Exception):
    """
    Custom exception for errors returned by SDXL 1.0.
    """

    def __init__(self, message):
        self.message = message


# Set up logging
logger = logging.getLogger(__name__)
if logger.hasHandlers():
    logger.handlers.clear()
handler = logging.StreamHandler()
logger.addHandler(handler)
formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
handler.setFormatter(formatter)
logger.setLevel(logging.INFO)


def main():
    build_request_body(
        MODEL_ID,
        POSITIVE_PROMPT,
        NEGATIVE_PROMPT,
    )


def generate_image(model_id, body):
    """
    Generate an image using SDXL 1.0 on demand.
    Args:
        model_id (str): The model ID to use.
        body (str) : The request body to use.
    Returns:
        image_bytes (bytes): The image generated by the model.
    """

    logger.info("Generating image with SDXL model %s", model_id)

    bedrock = boto3.client(service_name="bedrock-runtime",
                           region_name='us-west-2',
                           aws_access_key_id=userdata.get('AWS_ACCESS_KEY_ID'),
                           aws_secret_access_key= userdata.get('AWS_SECRET_ACCESS_KEY'))

    accept = "application/json"
    content_type = "application/json"

    response = bedrock.invoke_model(
        body=body, modelId=model_id, accept=accept, contentType=content_type
    )
    response_body = json.loads(response.get("body").read())
    logger.info(f"Bedrock result: {response_body['result']}")

    base64_image = response_body.get("artifacts")[0].get("base64")
    base64_bytes = base64_image.encode("ascii")
    image_bytes = base64.b64decode(base64_bytes)

    finish_reason = response_body.get("artifacts")[0].get("finishReason")

    if finish_reason == "ERROR" or finish_reason == "CONTENT_FILTERED":
        raise ImageError(f"Image generation error. Error code is {finish_reason}")

    logger.info("Successfully generated image with the SDXL 1.0 model %s", model_id)

    return image_bytes


def build_request_body(
    model_id,
    positive_prompt,
    negative_prompt,
):
    """
    Entrypoint for SDXL example.
    Args:
        model_id (str): The model ID to use.
        positive_prompt (str): The positive prompt to use.
        negative_prompt (str): The negative prompt to use.
    """

    # Build request body
    body = json.dumps(
        {
            "text_prompts": [
                {"text": positive_prompt, "weight": 1},
                {"text": negative_prompt, "weight": -1},
            ],
            "height": 512,
            "width": 512,
            "cfg_scale": 12,
            "clip_guidance_preset": "NONE",
            "sampler": "K_DPMPP_2M",
            "samples": 1,
            "seed": 123456,
            "steps": 50,
            "style_preset": "fantasy-art",
        }
    )

    # Generate and save image
    try:
        image_bytes = generate_image(model_id=model_id, body=body)
        image = Image.open(io.BytesIO(image_bytes))
        logger.info(image.show())
        logger.info(image.size)
        epoch_time = int(time.time())
        generated_image_path = f"{GENERATED_IMAGES}/image_{epoch_time}.png"
        logger.info(f"Generated image: {generated_image_path}")
        image.save(generated_image_path)
    except ClientError as err:
        message = err.response["Error"]["Message"]
        logger.error("A client error occurred: %s", message)
    except ImageError as err:
        message = err.response["Error"]["Message"]
        logger.error("A image error occurred: %s",message)

    else:
        logger.info(f"Finished generating image with SDXL model {model_id}.")


if __name__ == "__main__":
    main()

2024-03-10 01:14:54,485 - INFO - Generating image with SDXL model stability.stable-diffusion-xl-v1
INFO:__main__:Generating image with SDXL model stability.stable-diffusion-xl-v1
2024-03-10 01:14:59,882 - INFO - Bedrock result: success
INFO:__main__:Bedrock result: success
2024-03-10 01:14:59,889 - INFO - Successfully generated image with the SDXL 1.0 model stability.stable-diffusion-xl-v1
INFO:__main__:Successfully generated image with the SDXL 1.0 model stability.stable-diffusion-xl-v1
2024-03-10 01:15:00,012 - INFO - None
INFO:__main__:None
2024-03-10 01:15:00,017 - INFO - (512, 512)
INFO:__main__:(512, 512)
2024-03-10 01:15:00,020 - INFO - Generated image: /content/generated_images/image_1710033300.png
INFO:__main__:Generated image: /content/generated_images/image_1710033300.png
2024-03-10 01:15:00,147 - INFO - Finished generating image with SDXL model stability.stable-diffusion-xl-v1.
INFO:__main__:Finished generating image with SDXL model stability.stable-diffusion-xl-v1.
